---

# <u> **The Naïve, Barnes-Hut, and Fast Multipole Methods**  </u>
### *Implementation, Analysis, and Evaluation of different Algorithms for $N$-Body Simulations*  

**Author:** Hugo Robijns

**Date:** April 2025 

---

This notebook is written similarly to a typical formal write-up, and does not contain the bulk of the code, all of which can be found in the attached repository (see the README.md file for a detailed breakdown on the structure of the repository). Plots and tables (all made using Python by the author) are pre-generated since generating them on the spot would cause the notebook to run too slowly (N-body simulation is notoriously expensive).

___

### **Introduction:**
The analysis of many-body systems is highly relevant to many fields, from celestial bodies interracting gravitationally, to vortex methods in computational fluid dynamics, and more. These systems are complex to simulate: for three or more bodies the system becomes chaotic and extremely difficult to approach analytically without major simplifications, and naïvely calculating pairwise interractions for $N$ bodies leads to a numerical complexity of $\mathcal{O}(N^2)$, which scales poorly.  

&emsp; &emsp; This report aims to analyse and implement two algorithms which attempt to reduce this complexity, the Barnes-Hut and fast multipole method (FMM) techniques, alongside the brute-force $\mathcal{O}(N^2)$ approach, in order to compare their performances. For the sake of comparison, we will mainly base analysis around the simple 2D 'Coulomb gas', particles interacting under a logarithmic Coulombic potential.

&emsp; &emsp; The report begins with a section analysing the algorithms and their theoretical complexity, followed by a section describing their implementation, and finally a discussion section. We find, as expected from the literature, that FMM provides the biggest reduction in computational time for large systems, followed closely by the Barnes-Hut method. For smaller systems however (up to $10^3-10^4$ bodies) a simple vectorised naïve approach is the fastest, albeit with a high memory usage. 

---
## **1. Analysis of Algorithms**  


### *1.1 Naïve Pairwise Calculation, $\mathcal{O}(N^2)$*

In a 2D system, a particle $i$ of charge $q_i$ situated at $\vec{x}_i = (x_i,y_i)$ produces a electrostatic potential $\phi_{i}(x,y)$ and field $E_{i}(x,y)$ given by:
$$\phi_{i}(x,y) = -kq_i\log(|\vec{x}-\vec{x}_i|)$$
$$E_{i}(x,y) = -\nabla_{2\text{D}}\phi_{x_0}(x,y) = kq_i\frac{(\vec{x}-\vec{x}_i)}{|\vec{x}-\vec{x}_i|^2}$$
&emsp; &emsp; where $k$ is a constant. Therefore for an $N$-body system, we must sum over the fields produced by all the other particles, and the equations of motion become:
$$m_i \frac{\text{d}^2\vec{x}_i}{\text{d}t^2}  = q_i \sum_{j=1, j\neq i}^{N} \frac{q_j (\vec{x}_j-\vec{x}_i)}{|(\vec{x}_j-\vec{x}_i)|^2}$$ 

&emsp; &emsp; from which the position and velocity of the particles can be updated through integration.

It is clear to see that in order to simulate such a system directly, we must carry out $N-1$ calculations for each of the $N$ particles (or half of the particles, if exploiting the symmetry of Newton's $3^\text{rd}$ law), leading to a theoretical complexity of $\mathcal{O}(N^2)$. Note also that the force between two particles diverges as they get close to each other: this requires the inclusion of a softening parameter $\epsilon$ which will be discussed further in Section 2.1. 

### *1.2 Barnes-Hut Algorithm, $\mathcal{O}(N\log N)$*  
The Barnes-Hut Algorithm, introduced by Josh Barnes and Piet Hut in 1986 [1] attempts to reduce complexity by treating distant clusters of particles as a single body. It does this through hierarchical decomposition, in this case the use of tree data structures, specifically a quadtree in 2D. 

##### 1.2.1 Tree Construction
Beginning with a topmost node that represents the whole simulation space, a quadtree is formed by subdividing the space into 4 quadrants recursively until each body lies in its own leaf node, a node with no children (see Figure 2). Each node, both internal and leaf nodes, contains information about the charge distribution (specifically the total charge and centre of charge) of the node. Formally, they are constructed by inserting particles into the data structure one after the other:

1. if the node into which the particle is being placed is an empty leaf node, the particle is simply placed there and the total charge and centre of charge position of the node are trivially updated.

2. if the node is an internal node (i.e. a node with children), the total charge and centre of charge position of the node are updated, and we attempt to put the particle into the appropriate child.

3. if the node is a leaf node which already contains a particle, the total charge and centre of charge position of the node is updated, and the node is subdivided into 4 quadrants. Then, both particles are placed into their appropriate quadrants as above, which may require further subdivision if they end up in the same quadrant again.   


<img src="figures/quadtree_plot.png" width="99%" alt="figures/quadtree_plot.png" />

_Figure 1: A visualisation of the quadtree constructed for 10 particles in a 2D simulation. The square grid-lines seen in the left plot are the spatial borders of the leaf nodes, the smallest subdivisions of the tree which have no children of themselves, and therefore contain either 0 or 1 particles. The figure on the right shows a schematic of the quadtree: hollow markers indicate empty nodes, and solid markers indicate nodes containing particles._

##### 1.2.2 Force Calculation
To calculate forces, the tree is recursively traversed from the root for each particle:

1. If the node is a leaf node and contains a particle that is not itself, then the force is calculated in the usual way.

2. If the node is an internal node, the ratio: $$\frac{\text{size of node in space (i.e. side length of the cube)}}{\text{distance from particle to centre of charge of node}} \equiv \frac{s}{d} $$ is calculated. If this is less than a threshold value $\theta$ (usually taken to be $\sim$ 1), then the node is seen to contain particles that are closely packed and far away, and as a result can be treated as one single particle, ignoring its children nodes. The force on the particle is calculated in the usual way, using the centre of charge position and total charge of the node.

3. If $s/d > \theta$, the process recursively continues for each of the children of the node, until reaching the leaf node or a node which satisfies the above condition.

These forces are then summed to give an overall acceleration for the particle. It is clear to see that $\theta$ determines the accuracy of the simulation: a larger $\theta$ leads to fewer calculations but a more approximate solution, whilst a smaller $\theta$ may take longer but provide a more reliable solution. In the limit $\theta\rightarrow0$ the algorithm reduces to the brute-force approach, since no clusters are grouped together.

##### 1.2.3 Complexity
For a _balanced_ quadtree with $h$ levels, the bottom level has $4^h$ nodes. Therefore:
$$ 4^h \geq N \Rightarrow h \sim \log_4 N = \frac{\log_2 N}{\log_2 4} = \frac{1}{2} \log_2 N \text{ or } \mathcal{O}(\log N)$$
Insertion of each of the $N$ particles requires travelling the $h$ levels of the tree from root to leaf, therefore the overall complexity of constructing a quadtree is given by $\mathcal{O}(N\log N)$. 

The complexity of the force calculation step is more complicated. A typical reasoning (as laid out in the original paper) follows as such: again assuming a balanced quadtree (i.e. homogoneously distributed charge), consider quadrupling the number of particles. This is equivalent to adjoining 3 root nodes to the existing one. To calculate the force on a particle situated in the 'original tree', these additional particles will contribute a fixed number of extra calculations, which is a function of $\theta$ but not $N$. To imagine this, consider $\theta \gg 1$: this will simply add 3 force calculations, since the centre of charge of all three of the the 'additional root nodes' are far enough away, and they can be treated as a single particle. Reducing $\theta$ to more reasonable values will increase the number of children of these 'additional nodes' that need to be explored, but assuming $N$ is large enough (and this further exploration is therefore not reaching leaf nodes), then the number of additional calculations does not depend on $N$. Since the number of force calculations is increasing by an additive constant (a function of $\theta$) whilst $N$ is increasing by a multiplicative factor, the complexity goes as $\mathcal{O}(\log N)$ for a single particle, or $\mathcal{O}(N\log N)$ for $N$ particles. 

Summing the tree construction and force calculation complexities leads to overall complexity $\mathcal{O}(N\log N)$, a major improvement on the naive $\mathcal{O}(N^2)$.


### *1.3 Fast Multipole Method (FMM), $\mathcal{O}(N)$* 
The FMM algorithm was introduced by Leslie Greengard and Vladimir Rokhlin Jr. in the 1980s [2]. It uses similar hierarchical decomposition methods as the Barnes-Hut algorithm, but instead works with more accurate local and multipole expansions of the potential rather than simple information on the centre of charge of a node. Re-use of computation by traversing the tree in both directions allows complexity to be reduced to $\mathcal{O}(N)$ for arbitrary precision. For the sake of this report, detailed derivation of the maths and convergence properties will not be carried out: see the original paper for more detailed discussion. 

A high-level overview of the algorithm is as follows:

1. A quadtree is constructed as in the Barnes-Hut method, however the approach need not be as rigorous, since the algorithm simply requires leaf nodes to house a small, constant $\mathcal{O}(1)$ number of bodies.

2. The tree is traversed from bottom to top, calculating for each node the multipole expansion (the potential produced by the node at a point far away).

3. The tree is traversed from top to bottom, calculating for each node the local expansion (the potential felt at the node due to other nodes far away).

4. At the bottom leaf level, direct pair-wise interractions are calculated with nearest neigbours - since in step 1. we ensured each leaf node had a small number of bodies, this is inexpensive.

##### 1.3.1 Multipole Expansions and Translations
For $k=1$, expressing the logarithmic potential from before produced by a particle $i$ in terms of complex variables $(x,y) \in \mathcal{R}^2 \rightarrow z = x+iy \in \mathcal{C}$, we get:
$$\phi_{i}(x,y) \rightarrow \phi_{i}(z) = \text{Re}(-\log(z-z_0))$$
&emsp; &emsp; which for $|z|>|z_0|$ (and absorbing the minus sign into $q$) can be expanded as:
$$ \phi_{i}(z) = q \left(\log(z) - \sum_{k=1}^\infty \frac{1}{k} \left( \frac{z_0}{z} \right)^k \right) $$
Putting together the potential, as expressed above, from $N$ particles of charge $\{q_i, i = 1 \ldots N\}$ at positions $\{z_i, i = 1 \ldots N\}$ with $|z_i| < r$, gives us the _multipole expansion_ valid for $|z| > r$:
$$\phi(z) = Q\log(z) + \sum_{k=1}^\infty \frac{a_k}{z^k} \phantom{xxx}\text{ where } Q = \sum_{i=1}^{N} q_i \text{ and } a_k = \sum_{i=1}^{N}\frac{-q_iz_i^k}{k}$$
Although the infinite expansion converges and is therefore perfectly accurate, in reality we only use a finite number of terms, leading to a truncation error which since $|z_0| < |z|$, scales geometrically (error $\propto (|z_0|/|z|)^{-p}$).

The efficiency in the FMM algorithm comes in part due to its re-use of computation of the above expansion, through the use of translations. These allow expansions to be shifted to a different center, allowing coefficients of adjacent level nodes to be calculated more cheaply. The three relevant translations are the multipole-to-multipole shift (M2M translation, shifts the center of a multipole expansion):
$$ \phi(z) = Q \log (z) + \sum_{l=1}^\infty \frac{b^l}{z}  \phantom{xxx}\text{ where } b_l = \sum_{k=1}^l a_k z_0^{l-k} \binom{l-1}{k-1} -\frac{a_0 z_0^l}{l} $$
&emsp; &emsp; the multipole-to-local shift (M2L translation, shifts a multipole expansion to a local expansion):
$$ \phi(z) = \sum_{l=0}^{\infty}b_l z^l  \phantom{xxx}\text{ where } b_0 = \sum_{k=1}^\infty (-1)^k \frac{a_k}{z_0^k} + a_0 \log(-z_0) \text{ and } b_l = \frac{1}{z_0^l} \sum_{k=1}^\infty \frac{(-1)^k a_k}{z_0^k}\binom{l+k-1}{k-1} - \frac{a_0}{lz_0^l}$$
&emsp; &emsp; and the local-to-local shift (L2L translation, shifts the center of a local expansion):
$$ \sum_{k=0}^N a_k(z-z_0)^k = \sum_{l=0}^N \left(\sum_{k=l}^N a_k \binom{k}{l} (-z_0)^{k-l} \right)z^l$$


##### 1.3.2 Algorithm and Complexity
Starting from the leaf nodes, the multipole expansion for each node is calculated using the above analytical expression. Since we are essentially aggregating (summing) over the charges, this step is $\mathcal{O}(N)$. Going up the tree, the expansion of each node can be formed through aggregating the expansion of its children, using the M2M translation and summing. Again, since this is simply an aggregation over the charges, the whole upwards pass is $\mathcal{O}(N)$.

Working downards from the top of the tree, we calculate the local (Taylor) expansion of the potential experienced at each node. Firstly, the local expansion of the parent node (if it exists) is shifted to the centre of the new node using an L2L translation. Then, the multipole expansion of non-adjacent (far away) nodes is translated into a local expansion at the location of the node, using M2L translation. In 2D, this can only ever be less than 27 interactions, independent of $N$, (see Figure X) and therefore is a constant amount of work. Finally, upon reaching the bottom leaf level, we calculate the direct contributions from the few remaining bodies, which is not computationally expensive since there are a small number of bodies at this level. Overall, the work done in the downard pass is constant for each body, and thefore $\mathcal{O}(N)$ overall. 

<img src="figures/FMMinteractionlist.png" width="99%" alt="figures/FMMinteractionlist.png" />

_Figure 2: A demonstration of how the interaction list is determined for a specific node (green) in FMM. At each level, non-adjacent (purple) nodes are treated using M2L translations. Of the nodes that are too close (red), further division occurs (moving down a level in the tree) and the process repeats itself. Those that have already been dealt with (grey) are naturally passed down through the L2L translations from parents to kids. At the bottom level, the potentials from the remaining particles are summed naively, since there are only a small number left. Looking at levels 3 and 4, it is clear to see that in 2D, the interaction set is at most 27 (6x6 - 3x3), and therefore provides a constant amount of work, independant of $N$._


With an efficient quadtree implementation, this thereotically allows N-body simulation which is linear in N, a huge improvement on the quadratic naïve approach. Furthermore, since expansion order doesn't influence the linear scaling, the solution can be arbitrarily accurate (up to other limiting factors, such as machine precision).

---

## **2. Implementation and Performance**

All three algorithms were implemented in vanilla Python alongside some basic open-source libraries, mainly _Numpy_ and _Matplotlib_, and modules _time_ and _collections_. For timing comparison, the results in this report were produced by running the algorithms on a Macbook Air with CPU 1.1 GHz Quad-Core Intel Core i5. Note that, since the aim of the report was to compare performance and complexities, all simulations were carried out with arbitrary simulation units (Coulomb constant $k=1$, $q=1$ etc.). The README.md file gives information on where to find the relevant scripts, which are extensively commented. This section provides an overview of certain implementation choices that were made, a snippet of the main code, a few tests which show that the algorithms were implemented correctly, and a brief look at the time and memory complexity.

### *2.1 Naïve Pairwise Calculation (source/naive)*
The brute force implementation is found in source/naive. This algorithm is the simplest to implement, but its time complexity scales the worst with $N$. The following subsections discuss the softening parameter, time step and integration techniques, which are relevant for the other algorithms too.


##### 2.1.1 Softening Parameter ($\epsilon$) and Time Step ($\Delta t$)
In order to avoid divergence and instability when particles become close, we introduce a softening parameter $\epsilon$ such that:
$$\phi_{i}(x,y)=-kq_i\log(|\vec{x}-\vec{x}_i|)\phantom{x}\xrightarrow{\text{softening}} \phantom{x}\phi_{i}(x,y)=-kq_i\log(\sqrt{|\vec{x}-\vec{x}_i|^2 + \epsilon^2})$$ 
$$E_{i}(x,y) = -\nabla_{2\text{D}}\phi_{x_0}(x,y) = kq_i\frac{(\vec{x}-\vec{x}_i)}{|\vec{x}-\vec{x}_i|^2}\phantom{x}\xrightarrow{\text{softening}}\phantom{x} E_{i}(x,y)  = kq_i\frac{(\vec{x}-\vec{x}_i)}{|\vec{x}-\vec{x}_i|^2+\epsilon^2}$$
$\epsilon$ should be chosen to be small enough to keep the simulation accurate, but large enough to avoid singularities. This depends on the scale of the system, and the time-step, $\Delta t$. A demonstration of the influence of $\epsilon$ is shown below.

<img src="figures/softening.gif" width="99%" alt="figures/FMMinteractionlist.png" />

_Figure 3: a demonstration of the effect of the softening parameter $\epsilon$ showing an animation of three bodies interracting (left), and the percentage total energy change of the system from its initial value for three different values of $\epsilon$ (right). For smaller values of $\epsilon$, the close passes cause larger variations away from the initial, conserved energy of the system. However, for larger values of $\epsilon$, this level of conserved energy is further away from the reality. Note that for this simulation, in contrast to the others in this report, an attractive potential has been used to encourage close passes._

Work has been carried out in order to quantify the most reliable size of $\epsilon$ and $\Delta t$ for a system, often based on how far particles travel in a time-step in relation to their proximity. Indeed, a good way forward is to use an adapative $\Delta t$, which can become smaller as bodies become closer to each other, in order to accurately and stably explore regions of high accelaration. Since this project is less about dealing with collisions, and more about analysing the complexities and performances of the algorithms in scaling to large numbers of bodies, a simple comparison and tuning process was carried out, settling on constant values $\epsilon \sim 0.5$, and $\Delta t \sim 0.01$. These values were kept consistent and used for the other two algorithms as well. In addition, for our complexity analysis we consider systems wherein the particles are well-separated, to avoid any unnecessary instability.


##### 2.1.2 Integration Techniques
Upon calculating the individual acceleration each particle experiences, we integrate to find their updated velocity and position. There are many different possible integration techniques, but we used the symplectic 'kick-drift-kick' technique due to its easy implementation and improved energy conservation compared to the simple Euler step. It is a form of Verlet-velocity integration, and can be implemented through the following equations:


1. $x(t+\Delta t) =x(t)+v(t)\Delta t+\frac{1}{2}a(t)(\Delta t)^2$

2. calculate $a(t+\Delta t)$

3. $v(t+\Delta t) = v(t) + \frac{1}{2}\left(v(t)+v(t+\Delta t)\right)\Delta t$

&emsp; &emsp; where $x$ is position, $v$ is velocity, $a$ is acceleration, and $t$ is time. The figure below shows the energy drift over time for a simple Euler step compared to 'kick-drift-kick'. 


<img src="figures/integration.gif" width="99%" alt="figures/FMMinteractionlist.png" />

_Figure 4: A plot showing an animation of 75 bodies under a repulsive potential (left) and the associated energy conservation of the system (right) for two different integrators, Euler step and kick-drift-kick. As expected, kick-drift-kick conserves energy better than Euler. The error in an Euler step approach is theoretically linear over time: the tailing off seen is most likely due to acceleration (and therefore distance travelled each step, and associated energy drift) reducing as particles spread further apart over time._

##### 2.1.3 Implementation

The full implementation for the naive approach (source/naive/simulation.py) is shown below (imports are commented out since the code is not used in this notebook):

In [1]:
# import numpy as np

k = 1.0      # Coulomb constant (in appropriate simulation units)
soft = 0.1   # softening length
dt = 0.01    # time step 

class Body:
    """
    Represents a charged particle with position, velocity, charge, mass, and accumulated force.

    Attributes:
        position (np.ndarray): 2D position vector of the body.
        velocity (np.ndarray): 2D velocity vector of the body.
        charge (float): Electric charge of the body.
        mass (float): Mass of the body (default = 1.0).
        force (np.ndarray): Accumulated force vector acting on the body.
    """

    def __init__(self, position, velocity, charge, mass=1.0):
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.charge = charge
        self.mass = mass
        self.force = np.zeros(2)

class Simulation:
    """
    Brute-force calculation of pairwise Coulomb interactions.
    Uses the kick-drift-kick algorithm for time integration.
    """
    def __init__(self, bodies):
        # store a list of Body instances
        self.bodies = bodies

    def compute_forces(self):
        """Compute the Coulomb force on each body due to every other body."""
        # reset forces before accumulation
        for b in self.bodies:
            b.force = np.zeros(2)

        # loop over unique pairs (i, j) with i < j (avoids double counting)
        N = len(self.bodies)
        for i in range(N):
            for j in range(i + 1, N):
                b1 = self.bodies[i]
                b2 = self.bodies[j]

                # b2 -> b1
                diff = b1.position - b2.position
                r2 = np.dot(diff, diff)
                r_soft = np.sqrt(r2 + soft**2) # softening

                # 2D Coulomb force: F = k * q1 * q2 * r_vec / r^2
                f_vec = k * b1.charge * b2.charge * diff / (r_soft**2)

                # Newton's third law
                b1.force += f_vec
                b2.force -= f_vec

    def step(self):
        """
        Kick-drift-kick integration:
        1. Half-step velocity update: v += (F / m) * (dt/2)
        2. Full-step position update: x += v * dt
        3. Recompute forces at the new positions
        4. Half-step velocity update: v += (F / m) * (dt/2)
        """
        for b in self.bodies:
            b.velocity += 0.5 * (b.force / b.mass) * dt  # (1)
            b.position += b.velocity * dt                # (2)

        self.compute_forces() # (3)

        for b in self.bodies:
            b.velocity += 0.5 * (b.force / b.mass) * dt # (4)

##### 2.1.4 Complexity

<img src="figures/naivecomplexity.png" width="99%" alt="figures/naivecomplexity.png" />

_Figure 5: A plot showing time taken to compute a single step, averaged over 5 seeds (left) and the equivalent data tabulated, along with peak memory usage, for selected values of N (right). As expected, the naïve approach scales poorly with $N$, and the plot is (loosely) quadtratic. The 'wobbles' and spread are most-likely caused by cache-level effects and python interpreter overheads: once this approach was vectorised (see Section 3.2.2), a much smoother quadratic was observed. Indeed, plotting the number of force calculations as a function of $N$ gives a perfect quadratic._

### *2.2 Barnes-Hut (source/BH)*
The Barnes-Hut implementation is found in source/BH. The main difference compared with the naïve approach is the introduction of the quadtree for hierarchical decomposition: softening, force calculations (albeit different ones) and integration proceeds identically to before. The code below shows the main algorithm (source/BH/quadtree.py), which constructs and uses a quadtree to compute the forces in a more efficient manner.

In [ ]:
# import numpy as np

k = 1.0      # Coulomb constant (in appropriate simulation units)
soft = 1.0   # softening length
dt = 0.01    # time step 
theta = 0.5  # theta

class QuadTree:
    """QuadTree node for Barnes-Hut algorithm in 2D."""
    def __init__(self, x_min, x_max, y_min, y_max):
        # spatial bounds of this node
        self.x_min, self.x_max = x_min, x_max
        self.y_min, self.y_max = y_min, y_max
        # combined center-of-charge of contained bodies (2D vector)
        self.center_of_charge = np.zeros(2)
        # total charge of contained bodies
        self.total_charge = 0.0
        # Body instances it contains
        self.bodies = []
        # children it has, (if leaf, none)
        self.children = None

    def contains(self, pos):
        """
        Check whether a 2D position lies within this node's bounds.
        Returns True if x_min <= pos[0] < x_max and y_min <= pos[1] < y_max.
        """
        return (self.x_min <= pos[0] < self.x_max and
                self.y_min <= pos[1] < self.y_max)

    def insert(self, body):
        """
        Insert a charged Body into the tree:
        - If this node is an empty leaf, store the body here.
        - Otherwise, ensure children exist (subdivide if needed),
          update this node's total_charge and center_of_charge,
          and delegate insertion to the appropriate child.
        """
        # case 1: empty leaf node
        if self.total_charge == 0 and not self.bodies:
            self.bodies.append(body)
            self.center_of_charge = body.position.copy()
            self.total_charge = body.charge
            return

        # case 2: non-empty or internal node
        # subdivide this leaf if it hasn't been already
        if self.children is None:
            self._subdivide()

        # update this node's aggregated charge & center-of-charge
        old_q = self.total_charge
        self.total_charge += body.charge
        self.center_of_charge = (
            old_q * self.center_of_charge
            + body.charge * body.position
        ) / self.total_charge

        # delegate to one of the four children based on body position
        for child in self.children:
            if child.contains(body.position):
                child.insert(body)
                return

    def _subdivide(self):
        """
        Split this node's region into four quadrants and create child nodes.
        Then re-insert any existing body into the appropriate quadrant.
        """
        # compute midpoints
        xm = 0.5 * (self.x_min + self.x_max)
        ym = 0.5 * (self.y_min + self.y_max)
        # create four children: SW, SE, NW, NE quadrants
        self.children = [
            QuadTree(self.x_min, xm,    self.y_min, ym),  # SW
            QuadTree(xm,     self.x_max, self.y_min, ym),  # SE
            QuadTree(self.x_min, xm,    ym,         self.y_max),  # NW
            QuadTree(xm,     self.x_max, ym,         self.y_max),  # NE
        ]
        # re-insert the existing body (if any) into correct child
        for b in self.bodies:
            for c in self.children:
                if c.contains(b.position):
                    c.insert(b)
                    break
        # clear bodies list for this internal node
        self.bodies = []

    def compute_force(self, body, theta):
        """
        Recursively compute Coulomb force on 'body' from this node, 
        using BH theta criterion.

        Returns:
            np.ndarray: 2D force vector on 'body'.
        """
        # empty region or same body -> no force
        if self.total_charge == 0 or (len(self.bodies) == 1 and self.bodies[0] is body):
            return np.zeros(2)

        diff = body.position - self.center_of_charge
        r2 = diff.dot(diff)
        r = np.sqrt(r2 + soft**2) # softening
        width = self.x_max - self.x_min

        # BH criterion
        if width / r < theta or self.children is None:
            return k * body.charge * self.total_charge * diff / (r**2)
        else:
            # otherwise, sum forces from each child cell
            F = np.zeros(2)
            for c in self.children:
                F += c.compute_force(body, theta)
            return F

Using this script to calculate the forces, the integration and simulation proceeds as before. 

##### 2.2.1 Theta
A good verification to ensure the implementation is working correctly is observing how the model behaves as a function of $\theta$. We expect that reducing $\theta$ brings the model closer to the naïve approach, improving energy conservation, but also increasing computation time. These effects are shown in Figure 6 below.

<div style="display: flex; justify-content: space-between; align-items: center;">
  <img src="figures/theta_energy.png" width="49%" alt="theta_energy.png" />
  <img src="figures/theta_time.png" width="49%" alt="other_figure.png" />
</div>

_Figure 6: two plots showing energy conservation over time for different values of $\theta$ (left) and computation time - for a single step of 200 bodies - against $\theta$ (right). As expected, increasing $\theta$ leads to a (small) worsening of energy conservation, but also a reduction of computing time, and vice versa._

##### 2.2.1 Complexity


<img src="figures/BHcomplexity.png" width="99%" alt="figures/BHcomplexity.png" />

_Figure 7: A plot showing, for $\theta =0.5$, time taken to compute a single step, averaged over 5 seeds (left) and the equivalent data tabulated, along with peak memory usage, for selected values of N (right). As expected, the BH approach scales considerably better than the naive approach, but uses more memory due to the more complicated data structures used._

### *2.3 FMM (source/FMM)*
The FMM implementation is found in source/FMM. This algorithm is by far the most complex to implement and relies on both hierarchical decomposition (another quadtree), and careful manipulation of the various aforementioned expansions and translations during a double traversal of the tree. Separate scripts define the new quadtree (source/FMM/quadtree.py) and kernel functions (source/FMM/kernels.py).

In [ ]:
# import numpy as np
# from quadtree import QuadTree
# from kernels import multipole, M2M, M2L, L2L

def distance(p1, p2, soft=1e-3):
    """Softened distance between two points"""
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    r2 = dx*dx + dy*dy
    return np.sqrt(r2 + soft*soft)

class Point:
    """Point in 2D"""
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.position = (x, y)

class Particle(Point):
    """A charged particle with charge q and potential phi."""
    def __init__(self, x, y, charge):
        super().__init__(x, y)
        self.q = charge
        self.phi = 0.0

class FMM2D:
    """Fast Multipole Method in 2D using a quadtree and complex expansions."""
    def __init__(self, particles, max_points=2, nterms=5):
        self.particles = particles
        self.nterms = nterms
        xs = [p.x for p in particles]
        ys = [p.y for p in particles]
        boundary = (min(xs), min(ys), max(xs), max(ys))
        self.tree = QuadTree(self.particles, boundary=boundary, max_per_leaf=max_points)

    def upward_pass(self):
        """Compute multipole expansions (outer) from leaves up to the root."""
        def recurse(node):
            if node.is_leaf():
                node.outer = multipole(node.points, center=node.center, nterms=self.nterms)
            else:
                for child in node.children:
                    recurse(child)
                node.outer = np.zeros(self.nterms+1, dtype=complex)
                for child in node.children:
                    z0 = complex(*child.center) - complex(*node.center)
                    node.outer += M2M(child.outer, z0)
        recurse(self.tree.root)

    def downward_pass(self):
        """Compute local expansions (inner) from root down to leaves and evaluate at leaves."""
        root = self.tree.root
        root.inner = np.zeros(self.nterms+1, dtype=complex)
        
        def recurse(node):
            if node.parent is not None:
                # shift parent's local expansion
                z0 = complex(*node.center) - complex(*node.parent.center)
                node.inner = L2L(node.parent.inner, z0)
                # add contributions from well-separated cells
                for in_node in node.interaction_list:
                    z1 = complex(*node.center) - complex(*in_node.center)
                    node.inner += M2L(in_node.outer, z1)
            if node.is_leaf():
                self.evaluate(node)
            else:
                for child in node.children:
                    recurse(child)
        recurse(root)

    def evaluate(self, node):
        """Evaluate potentials for particles in a leaf node."""
        zc = complex(*node.center)
        for p in node.points:
            z = complex(*p.position)
            # far-field via local expansion
            phi = 0.0
            for l, coeff in enumerate(node.inner):
                phi += coeff * (z - zc)**l
            # near-field via direct computation
            for neighbor in node.neighbors + [node]:
                for s in neighbor.points:
                    if s is not p:
                        r = distance(p.position, s.position)
                        phi += s.q * np.log(r)
            p.phi = phi.real if np.iscomplexobj(phi) else phi

    def compute(self):
        """Perform FMM and return potentials in input order."""
        # reset potentials
        for p in self.particles:
            p.phi = 0.0
        # upward and downward passes
        self.upward_pass()
        self.downward_pass()
        return np.array([p.phi for p in self.particles])

def potential(particles, tree_thresh=2, nterms=5):
    """
    Convenience function: build FMM2D and compute potentials.
    :param particles: list of Particle objects
    :param tree_thresh: max bodies per leaf
    :param nterms: number of expansion terms
    """
    fmm = FMM2D(particles, max_points=tree_thresh, nterms=nterms)
    return fmm.compute()

##### 2.3.1 Expansion Order

<div style="display: flex; justify-content: space-between; align-items: center;">
  <img src="figures/FMMerror.png" width="52%" alt="FMMerror.png" />
  <img src="figures/FMMtiming_p.png" width="47%" alt="FMMtiming_p.png" />
</div>

_Figure 8: As explained in section 1.3, increasing expansion order should increase the accuracy of the solution, but since we are increasing the number of calculations we are doing, also increase computational time. This can be seen here, where we have plotted error (relative difference compared to the brute force approach) against expanion order (left), and computation time for a step with 100 bodies as a function of expansion order (right)_

##### 2.3.1 Complexity

<img src="figures/FMMcomplexity.png" width="99%" alt="figures/BHcomplexity.png" />

_Figure 9: A plot of computation time of one step against number of bodies (left) and the equivalent data, along with memory usage, tabulated for certain values of $N$ (right). The scaling can be seen to be roughly linear._

___

## **3. Results and Discussion**
All three models are successful in simulating N-body 2D Coulombic systems. This section seeks to compare the models, their advantages and disadvantages, applications to certain problems and areas for further work.

### _3.1 Accuracy (conservation of energy, linear momentum and angular momentum)_
As discussed, the naive and FMM approaches have the capacity to perfectly conserve energy. For an FMM implementation of high expansion order, we can have a highly accurate solution that still scales linearly, and is therefore faster than the naive approach. The difficult with the Barnes-Hut method is that, although increasing $\theta$ allows us to speed up the computation, we also lose accuracy. It is also important to note that the naive approach by definition conserves momentum, since we include in our code a sum which ensures no double counting, and simply applies Newton's third law to give the two bodies involved an equal and opposite force (see Section 2.1.3). The same is not naturally true for BH and FMM methods. 

### _3.2 Extensions of the Models (3D problems, vecorisation etc.)_

##### 3.2.1 Different Problems and 3D
Extending the 2D simulation to 3D is very simple for the naive and BH approaches, the latter simply requiring the construction of an octree rather than a quadtree, whilst all the logic stays the same. For FMM however, the algorithm becomes much more complicated for a variety of reasons. To name a few: the 3D expansion now comes in the form of spherical harmonics, and for order $p$ there are $(p+1)^2$ coefficients; translations require the evaluation of non-trivial recurrence relations and complex coupling matrices; and interaction sets can be far larger.

In our current implementation, the BH approach uses an adaptive quadtree, whilst for simplicity our FMM uss a symmetric one. This means that our BH approach is better at dealing with non-uniform particle distributions. This is not an inherent difference between the algorithms, however: an avenue for further work would be to introduce an adaptive quadtree to our FMM implementation (although this is not trivial since care must be taken with the interraction list in an asymmetric tree). 

##### 3.2.2 Vectorisation
The code in this report is meant to be a clear and intuitive implementation of the algorithms: in reality, they can be made a lot faster. A first simple way is to vectorise the Python loops using NumPy array broadcasting. This is very simple to implement for the naive approach (see source/naive/simulation.py), and at the cost of a large memory increase (from allocating the $N\times N$ arrays), greatly speeds up computation:

In [ ]:
class SimulationVectorised:
    """
    Vectorised implementation of brute force approach using NumPy broadcasting.
    Reduces Python loops by operating on whole arrays.
    """
    def __init__(self, pos, vel, charge, mass=None):
        # pos: array of shape (N, 2), vel: array of shape (N, 2), charge: array of length N
        self.pos = np.array(pos, dtype=float) # (N,2)
        self.vel = np.array(vel, dtype=float) # (N,2)
        self.charge = np.array(charge, dtype=float) # (N,)
        self.N = self.pos.shape[0]
        self.force = np.zeros_like(self.pos) # (N,2)
        self.mass = np.ones(self.N) if mass is None else np.array(mass, dtype=float)

    def compute_forces(self):
        """
        Compute forces without explicit loops"""
        # pairwise position differences: (N, N, 2)
        diff = self.pos[:, None, :] - self.pos[None, :, :]
        # squared distances plus softening: (N, N)
        r2 = np.sum(diff * diff, axis=2) + soft**2
        # inverse squared distances
        inv_r2 = 1.0 / r2
        # zero diagonal to remove self-force
        np.fill_diagonal(inv_r2, 0.0)

        # outer product of charges: (N, N)
        qq = np.outer(self.charge, self.charge)
        # force contributions tensor: (N, N, 2)
        Fmat = k * qq[:, :, None] * diff * inv_r2[:, :, None]
        # sum over j to get net force on each i: (N, 2)
        self.force = np.sum(Fmat, axis=1)

    def step(self):
        """ kick-drift-kick as before """
        self.vel += 0.5 * (self.force / self.mass[:, None]) * dt # (1)
        self.pos += self.vel * dt # (2)
        self.compute_forces() # (3)
        self.vel += 0.5 * (self.force / self.mass[:, None]) * dt # (4)

Much of the computation in FMM is also easily vectorised, specifically constructing and translating the expansion coefficients. For BH, however, it is more complicated as the work mainly involves traversing a tree. 

### _3.3 Time Complexity Comparison_
One of the main objectives of this report was to compare how the complexity of the different methods scales with $N$. A summary figure is presented below.

<img src="figures/timingcomparison.png" width="99%" alt="figures/timingcomparison.png" />

_Figure 10: A time comparison between 4 different approaches to N-body simulation (FMM, BH, vanilla naive and vectorised naive). As expected, for high $N$, FMM and BH have lower a computation time per step than the naive approaches. However, due to their complexities (data structures constructed etc.) take longer at lower $N$: a simple vectorised brute force approach is faster than the intricate FMM till $N$ reaches nearly $10^4$._


### _3.4 Conclusion_
Overall, the three models have their advantages and disadvantages in certain situations. A key factor in deciding which model to use should also be their difficulty of implementation: although FMM provides good energy conservation, complexity scaling and can be applied to a whole host of problems, it is by far the most complicated to implement. Various libraries do exist (fmmlib2d, fmmlib3d etc.) but due to the complexity of the problem are often writen in Fortran and are not particularly accessible and adaptable to a desired problem. Meanwhile, a BH approach is far simpler to implement, can provide a reasonable solution, and in this case scales similarly to FMM. Although often quick to be neglected for the other algorithms, for medium-sized simulations (up to $10^3-10^4$), a simple vectorised brute-force approach can provide an accurate and comparable if not faster solution.   

---

## Bibliography  

1. J. Barnes and P. Hut, *A Hierarchical O(N log N) Force-Calculation Algorithm*, Nature, 1986.  

2. L. Greengard and V. Rokhlin, *A Fast Algorithm for Particle Simulations*, Journal of Computational Physics, 1987. 

<a id='bibliography'></a>